In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='3'
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# from transformers import PaliGemmaProcessor, PaliGemmaForConditionalGeneration
from src.processing_paligemma import PaliGemmaProcessor
from src.modeling_paligemma import PaliGemmaForConditionalGeneration
import torch
from torch.utils.data import DataLoader
from src.data import ClevrerDataset
import torchvision.transforms as transforms
import numpy as np

# --- Setup ---
model_id = "google/paligemma2-3b-mix-448"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# activations_dir = "../extracted_activations"  # Directory to save activations
# os.makedirs(activations_dir, exist_ok=True)

# Load model and processor
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id,
                                                          device_map="auto",
                                                        #   device_map=device,
                                                          attn_implementation="eager",
                                                          token_compression=None,
                                                          target_length=None).eval()
processor = PaliGemmaProcessor.from_pretrained(model_id)

# --- Load a small subset of the CLEVRER dataset for demonstration ---
frames_root = "../train_frames/"  # Replace with the actual path to your CLEVRER frames
json_path = "../miscellaneous/train.json"  # Replace with the actual path to your CLEVRER annotations


# Load the full dataset
full_dataset = ClevrerDataset(frames_root=frames_root, json_path=json_path, transform=None)



/data/ai24mtech02001/miniconda3/envs/physic/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/data/ai24mtech02001/miniconda3/envs/physic/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
# Create a small subset (e.g., first sample) for demonstration
subset_indices = [10]

print(subset_indices)

subset_dataset = torch.utils.data.Subset(full_dataset, subset_indices)
dataloader = DataLoader(subset_dataset, batch_size=1, shuffle=False)

item = next(iter(dataloader))


[10]


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [5]:
item

{'frames': [tensor([[[[0.4000, 0.4039, 0.4078,  ..., 0.4275, 0.4314, 0.4353],
            [0.4000, 0.4000, 0.4039,  ..., 0.4275, 0.4314, 0.4353],
            [0.4000, 0.4000, 0.4039,  ..., 0.4275, 0.4314, 0.4353],
            ...,
            [0.5412, 0.5412, 0.5412,  ..., 0.5137, 0.5137, 0.5137],
            [0.5412, 0.5412, 0.5412,  ..., 0.5137, 0.5137, 0.5137],
            [0.5412, 0.5412, 0.5412,  ..., 0.5137, 0.5137, 0.5137]],
  
           [[0.4000, 0.4039, 0.4078,  ..., 0.4275, 0.4314, 0.4353],
            [0.4000, 0.4000, 0.4039,  ..., 0.4275, 0.4314, 0.4353],
            [0.4000, 0.4000, 0.4039,  ..., 0.4275, 0.4314, 0.4353],
            ...,
            [0.5373, 0.5373, 0.5373,  ..., 0.5176, 0.5176, 0.5176],
            [0.5373, 0.5373, 0.5373,  ..., 0.5176, 0.5176, 0.5176],
            [0.5373, 0.5373, 0.5373,  ..., 0.5176, 0.5176, 0.5176]],
  
           [[0.4000, 0.4039, 0.4078,  ..., 0.4275, 0.4314, 0.4353],
            [0.4000, 0.4000, 0.4039,  ..., 0.4275, 0.4314, 0.435

In [8]:
question = "<image> "+item["question"][0]

In [9]:

model_inputs = processor(text=question, images=[item["frames"]], return_tensors="pt").to(torch.bfloat16).to(model.device)

ValueError: axes don't match array

In [4]:
out = model(**model_inputs)

******************************
Image features shape: torch.Size([8, 128, 2304])
Input embeds shape: torch.Size([1, 8204, 2304])


/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:371: masked_scatter_size_check: block: [0,0,0], thread: [0,0,0] Assertion `totalElements <= srcSize` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [8]:
import importlib
importlib.reload(importlib.import_module('src.activation_manager'))
from src.activation_manager import ActivationReplacementExperiment

# --- Initialize ActivationReplacementExperiment ---
activation_experiment = ActivationReplacementExperiment(model, processor, device=device, activations_dir=activations_dir)


In [4]:
# --- Define the layer(s) to extract activations from ---
layer_names_to_extract = [
        # 'vision_tower.vision_model.encoder.layers.5.self_attn',
        # 'vision_tower.vision_model.encoder.layers.4.self_attn.k_proj',
        'language_model.model.layers.15.self_attn.q_proj',
        'language_model.model.layers.15.self_attn.k_proj',
        'language_model.model.layers.15.self_attn.v_proj',
           ]


# --- Extract 'real' activations ---
activation_experiment._extract_activations(dataloader, layer_names_to_extract, store_as='real')

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


ValueError: axes don't match array

: 

In [ ]:
# --- Save the extracted activations to a text file ---
output_file = os.path.join(activations_dir, "real_activations.txt")

for layer_name in layer_names_to_extract:  # Iterate only through the layers we extracted
        if layer_name in activation_experiment.real_activations:
            activations = activation_experiment.real_activations[layer_name]
            # activations = activations.cpu().numpy().tolist()
            print(activations.shape)


# with open(output_file, "w") as f:
#     for layer_name in layer_names_to_extract:  # Iterate only through the layers we extracted
#         if layer_name in activation_experiment.real_activations:
#             f.write(f"Layer: {layer_name}\n")
#             activations = activation_experiment.real_activations[layer_name]
#             activations = activations.cpu().numpy().tolist()
#             f.write(f"    {activations}\n")
#             f.write("\n")

# print(f"Extracted activations from specified layers saved to: {output_file}")

torch.Size([1, 8204, 2048])
torch.Size([1, 8204, 1024])
torch.Size([1, 8204, 1024])


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='3'
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
from src.data import ClevrerDataset

test_frames_dir = "../test_frames"
annotations_path = "../miscellaneous/validation.json"

dataset = ClevrerDataset(
    frames_root=test_frames_dir,
    json_path=annotations_path,
    question_type="predictive",
    transform=None
)

train_ds, test_ds = dataset.train_test_split(test_size=10)

/data/ai24mtech02001/miniconda3/envs/physic/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/data/ai24mtech02001/miniconda3/envs/physic/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Splitting 3557 unique videos: 3547 for train, 10 for test.
Split generated and saved to cache: miscellaneous/split_indices.json


In [3]:
len(test_ds)

10

In [9]:
test_ds[9]

{'frames': [<PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>],
 'question_id': 12,
 'question_type': 'predictive',
 'question': 'What will happen next? a) The gray object collides with the sphere b) The gray cylinder collides with the yellow cylinder',
 'answer': 'a',
 'video_filename': 'video_12738.mp4'}

In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='3'
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# from transformers import PaliGemmaProcessor, PaliGemmaForConditionalGeneration
from src.processing_paligemma import PaliGemmaProcessor
from src.modeling_paligemma import PaliGemmaForConditionalGeneration
import torch
from torch.utils.data import DataLoader
from src.data import ClevrerDataset
import torchvision.transforms as transforms
import numpy as np
import yaml
from peft import PeftModel, PeftConfig
# --- Setup ---


pretrained_path = "artifacts/Caravaggio_1571"
pretrained_path = os.path.join("/data1/ai24mtech02001/projects/mech_interp_physics_reasoning", pretrained_path)

config_path = os.path.join(pretrained_path, "config.yaml")

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)



model_id = "google/paligemma-3b-mix-224"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# activations_dir = "../extracted_activations"  # Directory to save activations
# os.makedirs(activations_dir, exist_ok=True)

# Load model and processor
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id,
                                                          device_map="auto",
                                                        #   device_map=device,
                                                          attn_implementation="eager",
                                                          # token_compression=config["model_train"]["token_compression"],
                                                          # target_length=config["model_train"]["target_length"]
                                                          ).eval()
processor = PaliGemmaProcessor.from_pretrained(model_id)


# lora_model_path = "/data1/ai24mtech02001/projects/mech_interp_physics_reasoning/artifacts/Caravaggio_1571/checkpoint-8000"
# model = PeftModel.from_pretrained(model, lora_model_path)
model.eval()


# # --- Load a small subset of the CLEVRER dataset for demonstration ---
# frames_root = "../train_frames/"  # Replace with the actual path to your CLEVRER frames
# json_path = "../miscellaneous/train.json"  # Replace with the actual path to your CLEVRER annotations


# # Load the full dataset
# full_dataset = ClevrerDataset(frames_root=frames_root, json_path=json_path, transform=None)



Loading checkpoint shards: 100%|██████████| 3/3 [00:46<00:00, 15.39s/it]


PaliGemmaForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(256, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=115

In [2]:
# --- Load a small subset of the CLEVRER dataset for demonstration ---
frames_root = "../test_frames/"  # Replace with the actual path to your CLEVRER frames
json_path = "../miscellaneous/validation.json"  # Replace with the actual path to your CLEVRER annotations


# Load the full dataset
full_dataset = ClevrerDataset(frames_root=frames_root, 
                              json_path=json_path,
                              question_type="predictive", 
                              transform=None)

train_ds, test_ds = full_dataset.train_test_split(test_size=10)

len(test_ds)

10

In [3]:
test_ds[0]

{'frames': [<PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>,
  <PIL.Image.Image image mode=RGB size=480x320>],
 'question_id': 13,
 'question_type': 'predictive',
 'question': 'Which event will happen next? a) The metal cube collides with the purple cylinder b) The metal cylinder and the purple object collide',
 'answer': 'b',
 'video_filename': 'video_10763.mp4'}

In [ ]:
from PIL import Image

# text = "Select all that apply. "+ test_ds[0]["question"] + "<image> " * int(8 * config["model_train"]["target_length"]/1024)
# text = "<image> " * 4 + "Name 5 colors"
# text = "answer en <image> <image> How many objects are in these images?"
text = "answer en \
Frame 1: <image> \
Frame 2: <image> \
How many objects have entered Frame 2?\n"
print(text)

image_path_1 = "/data1/ai24mtech02001/projects/mech_interp_physics_reasoning/train_frames/video_00000-01000/video_00000/frame_0.jpg"
image_path_2 = "/data1/ai24mtech02001/projects/mech_interp_physics_reasoning/train_frames/video_00000-01000/video_00000/frame_1.jpg"
image_1 = Image.open(image_path_1)
image_2 = Image.open(image_path_2)

# inputs = processor(images=[test_ds[0]["frames"]], text=text, return_tensors="pt").to(model.dtype).to(model.device)
# inputs = processor(images=[image_1], text=text, return_tensors="pt").to(model.dtype).to(model.device)
inputs = processor(images=[[image_1, image_2]], text=text, return_tensors="pt").to(model.dtype).to(model.device)

answer en Frame 1: <image> # Frame 2: <image> How many objects have entered Frame 2?



In [8]:
inputs

{'input_ids': tensor([[ 13072,    659, 235248,  ...,   2416, 235336,    108]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[[[-0.2000, -0.1922, -0.1843,  ..., -0.1451, -0.1373, -0.1294],
          [-0.2000, -0.2000, -0.1922,  ..., -0.1451, -0.1373, -0.1294],
          [-0.2000, -0.2000, -0.1922,  ..., -0.1451, -0.1373, -0.1294],
          ...,
          [ 0.0824,  0.0824,  0.0824,  ...,  0.0275,  0.0275,  0.0275],
          [ 0.0824,  0.0824,  0.0824,  ...,  0.0275,  0.0275,  0.0275],
          [ 0.0824,  0.0824,  0.0824,  ...,  0.0275,  0.0275,  0.0275]],

         [[-0.2000, -0.1922, -0.1843,  ..., -0.1451, -0.1373, -0.1294],
          [-0.2000, -0.2000, -0.1922,  ..., -0.1451, -0.1373, -0.1294],
          [-0.2000, -0.2000, -0.1922,  ..., -0.1451, -0.1373, -0.1294],
          ...,
          [ 0.0745,  0.0745,  0.0745,  ...,  0.0353,  0.0353,  0.0353],
          [ 0.0745,  0.0745,  0.0745,  ...,  0.0353,  0.0

In [12]:
model(**inputs)

PaliGemmaCausalLMOutputWithPast(loss=None, logits=tensor([[[ -7.9757,  -1.3099, -14.6626,  ...,  -5.6320,  -5.6418,  -5.6357],
         [ -9.7266,   5.2611, -13.3249,  ...,  -6.4588,  -6.4680,  -6.4640],
         [-12.6550,  -2.2570, -21.8329,  ...,  -8.2846,  -8.2754,  -8.2684],
         ...,
         [ -7.2076,   3.7804, -13.0085,  ...,  -4.3190,  -4.3376,  -4.3337],
         [-13.2116,  20.4997, -11.7819,  ...,  -6.1259,  -6.1417,  -6.1394],
         [ -7.3759,   3.8437,  -9.0900,  ...,  -5.9371,  -5.9442,  -5.9315]]],
       device='cuda:0'), past_key_values=<transformers.cache_utils.HybridCache object at 0x7b4d4cad7a90>, hidden_states=None, attentions=None, image_hidden_states=tensor([[[ 1.5772e-05, -9.4527e-03, -5.5193e-03,  ..., -3.9842e-03,
          -5.3947e-03, -7.4642e-03],
         [ 9.2647e-04,  2.0819e-03, -1.4647e-03,  ..., -9.3158e-04,
          -7.1842e-03,  5.3776e-03],
         [-1.1814e-03, -3.9834e-03,  2.0918e-04,  ..., -3.9332e-03,
          -6.0337e-03,  3.5331e

In [32]:
input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    print(generation.shape)
    print(generation)
    generation = generation[0]
    # generation = generation[0][input_len:]
    decoded = processor.decode(generation, skip_special_tokens=True)
    print(decoded)

torch.Size([1, 537])
tensor([[ 13072,    659,  17400, 235248, 235274, 235292, 235248, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
         257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152, 257152,
       

In [22]:
generation = generation[input_len:]
decoded = processor.decode(generation)
print(decoded)
generation

tensor([], dtype=torch.int64)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='3'

from huggingface_hub import hf_hub_download
import torch
from transformers import LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor

# Load the model in half-precision
model = LlavaNextVideoForConditionalGeneration.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf", torch_dtype=torch.float16, device_map="cuda")
processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")

# Load the video as an np.array, sampling uniformly 8 frames (can sample more for longer videos)
video_path = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")

conversation = [
    {

        "role": "user",
        "content": [
            {"type": "text", "text": "Why is this video funny?"},
            {"type": "video", "path": video_path},
            ],
    },
]

inputs = processor.apply_chat_template(conversation, num_frames=8, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt").to("cuda")

out = model.generate(**inputs, max_new_tokens=60)
processor.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.31s/it]


['USER: \nWhy is this video funny? ASSISTANT: The humor in this video comes from the unexpected and endearing nature of the situation. The baby is wearing glasses and appears to be reading a book, which is a humorous and endearing sight because babies are typically not expected to be able to read at such a young age.']